<a href="https://colab.research.google.com/github/mihailrc/computerVisionWorkshop/blob/master/PotholeTrainingYOLOv7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train Yolov7 on a Pothole Custom Dataset**

We will go through the steps to train yolov7 on a custom dataset like potholes. You can follow the steps with any other custom dataset as well.

# **Tutorial**

We will be implementing the following steps:

1. Clone repo and setup dependencies
2. Load custom dataset from Roboflow in YOLOv7 format
2. Run YOLOv7 training
3. Evaluation
4. Download sample **video** and run YOLOv7 inference on it.

# 1 Clone repo and set up dependencies

_(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)_

In [ ]:
%%bash
cd /content
git clone --recursive https://github.com/mihailrc/computerVisionWorkshop.git
cd computerVisionWorkshop
pip install -r requirements.txt

# 2 Load custom dataset from Roboflow in YOLOv7 format

Next, we'll download our dataset in the right format. Use the `YOLOv7 PyTorch` export. Note that this model requires YOLO TXT annotations, a custom YAML file, and organized directories. The roboflow export writes this for us and saves it in the correct spot.

If you are using a custom data set then follow the instructions below, other wise just use the code in the next line.

- make sure you sign up at https://roboflow.com/
- go to https://universe.roboflow.com/veldi-aashrith/potholes-yhssy/dataset/3 (This the custom dataset i used. you can pick and choose your own)
- click on the YOLOv7 box, check the option "show download code" and click Continue
- copy the code generated and use it as below

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Y7Yt5PctObeoJtGRULfo")
project = rf.workspace("veldi-aashrith").project("potholes-yhssy")
dataset = project.version(3).download("yolov7")

# 3 Run YOLOv7 Training

We're ready to start custom training.

In [ ]:
# download COCO starting checkpoint
%cd /content/computerVisionWorkshop/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [ ]:
# run this cell to begin training
%cd /content/computerVisionWorkshop/yolov7
!python train.py --batch 16 --epochs 55 --data {dataset.location}/data.yaml --weights 'yolov7_training.pt' --device 0 

# 4 Evaluation

We can evaluate the performance of our custom training using the provided evaluation script.

In [ ]:
# Run evaluation
!python detect.py --weights runs/train/exp/weights/best.pt --conf 0.1 --source {dataset.location}/test/images

# 5 Download sample **video** and run YOLOv7 inference on it.

In [ ]:
!pip install youtube_dl

import youtube_dl

link = 'https://www.youtube.com/embed/BQo87tGRM74?start=270&end=285'

ydl_opts = {}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    video_title = info_dict.get('title', None)

path = 'potholes.mp4'

ydl_opts.update({'outtmpl':path})

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([link])

In [ ]:
!pip install ffmpeg-python

import ffmpeg
(
    ffmpeg
    .input('/content/potholes.mp4')
    .trim(start=270, end=285)
    .setpts('PTS-STARTPTS')
    .output(f'/content/potholesshort.mp4')
    .run()
)

In [ ]:
%%bash
cd /content/computerVisionWorkshop/yolov7/
python detect.py --weights runs/train/exp/weights/best.pt --conf 0.25 --img-size 640 --source /content/potholesshort.mp4